In [3]:
import numpy as np
import sys
sys.path.append('/data01/AMPolyCalc/code')
from void_params import fit_ellipsoid, _edge_map
from rw_utils import read_raw_data_1X
from params import pixel_size_1X as pixel_size
from void_mapping import void_map_gpu


In [27]:
projs, theta, center, dark, flat = read_raw_data_1X("1", "1")
b = 4
voids_4 = void_map_gpu(projs, theta, center, dark, flat, b, pixel_size)
#voids_4.sort_by_size(reverse=True)

	TIME: RECONSTRUCTION 2.67 secs
	STAT: voids found - 34242
	TIME: LABELING 12.26 secs


In [26]:
#Creating chunked x_voids to put into multiprocessor
nv = len(voids_4["x_voids"])
nproc = 32

x_voids = voids_4["x_voids"]
sizes = voids_4["sizes"]

idx = np.argsort(sizes)
print(idx)
idx_old = np.arange(len(sizes))
print(idx_old)
idx_old = idx_old[idx]
print(idx_old)

idxs = np.array_split(idx, nproc)

x_voids_nested = [[x_voids[ii] for ii in idxs[jj]] for jj in range(nproc)]

#Return major axis length that corresponds to x_voids 
rad_max_nested = [np.random.normal(0,1,len(x_voids_nested[i])) for i in range(nproc)]

rad_max = np.concatenate(rad_max_nested, axis = 0)
rad_max = rad_max[idx_old]

print(rad_max)

[34241 34206 34205 ...     2     1     0]
[    0     1     2 ... 34239 34240 34241]
[34241 34206 34205 ...     2     1     0]
[-0.85799288  0.68856321  1.59442479 ... -0.26251539 -0.96451105
  0.53912733]


In [ ]:
def max_ellip_rad(x_voids):
    sizes = np.asarray([np.sum(x_voids[ii]) for ii in range(len(x_voids))])
    x_voids = [_edge_map(x_void) for x_void in x_voids]

    cen, rad, rot, loc_exists, ellipticity_E, ellipticity_P = fit_ellipsoid(x_voids,sizes)

    rad = [max(length) for length in rad]
    
    return rad


p = Pool(processes = 32)
OutList = p.starmap(max_ellip_rad, voids_4["x_voids"])
p.close()

print(OutList)